In [1]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import requests
import time
import datetime
import pymysql
import pymysql.cursors
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
url = 'https://kr.api.riotgames.com/'
api_key = 'RGAPI-9e3787f3-fa47-48e0-90dc-d72622bae337'
api = '?api_key='+api_key

In [3]:
host = '127.0.0.1'
port = '3306'
sid = 'League Of Legends'

user = 'root'
password = 'tmddk720'

## Upload dataframe to Database

In [4]:
def upload_df_to_db(host, port, sid, user, password, table, df, if_exists='append'):
    
    connection = pymysql.connect(host=host,
                                user=user,
                                password=password,
                                db=sid,
                                charset='utf8',
                                cursorclass=pymysql.cursors.DictCursor)

    engine = create_engine("mysql+pymysql://{user}:{password}@{host}/{db}"
                        .format(user=user, password=password, host=host, db=sid))


    try:
        with connection.cursor() as cursor:
#             print('upload')
            df.to_sql(name=table, con=engine, if_exists=if_exists, index=False)
            connection.commit()
#             print('upload db')
    
    except Exception as e:
        print(e)
                
    finally:
        connection.close()

In [5]:
def update_db(host, port, sid, user, password, sql, values, option='one'):
    
    connection = pymysql.connect(host=host,
                                user=user,
                                password=password,
                                db=sid,
                                charset='utf8',
                                cursorclass=pymysql.cursors.DictCursor)

    try:
        with connection.cursor() as cursor:
#             print('update')
            if option == 'one':
                cursor.execute(sql, values)
            elif option == 'many':
                cursor.executemany(sql, values)
                
            connection.commit()
#             print('update db')
    
    except Exception as e:
        print(e)
                
    finally:
        connection.close()

## get data from DB

In [6]:
def get_league_queue_from_db(tier):
    host = '127.0.0.1'
    port = '3306'
    sid = 'League Of Legends'

    user = 'root'
    password = 'tmddk720'
    
    con = pymysql.connect(host=host, user=user, password=password, db=sid, charset='utf8')
    
    try:
        league_queue_db = pd.read_sql("SELECT * FROM league_queue where tier = '" + tier + "'", con=con)
    except Exception as e:
        print(e)
    finally:
        con.close()
        
    return league_queue_db

In [7]:
def get_summoner_from_db(summonerId):
    host = '127.0.0.1'
    port = '3306'
    sid = 'League Of Legends'

    user = 'root'
    password = 'tmddk720'
    
    con = pymysql.connect(host=host, user=user, password=password, db=sid, charset='utf8')
    
    try:
        summoner_db = pd.read_sql("SELECT * FROM summoners where summonerId = '" + summonerId + "'", con=con)
    except Exception as e:
        print(e)
    finally:
        con.close()
        
    return summoner_db

In [8]:
def get_recent_gameId_from_db(summonerId):

    host = '127.0.0.1'
    port = '3306'
    sid = 'League Of Legends'

    user = 'root'
    password = 'tmddk720'
    
    con = pymysql.connect(host=host, user=user, password=password, db=sid, charset='utf8')
    sql = """SELECT gameId FROM `League Of Legends`.matchlists t1 
                WHERE t1.timestamp = (SELECT t2.timestamp FROM `League Of Legends`.matchlists t2 
                                        WHERE t2.summonerId = '%s' 
                                        ORDER BY t2.timestamp DESC LIMIT 1);""" %(summonerId)
    try:
        with con.cursor() as cursor:
            cursor.execute(sql)
            fetch = cursor.fetchall()
            
            if len(fetch) == 0:
                gameId = np.nan
            else:
                gameId = int(fetch[0][0])
    except Exception as e:
        print(e)
    finally:
        con.close()
    
    return gameId

In [9]:
#queue=420 : 소환사의 협곡
def get_gameId_list_from_db(queue='420', tier='CHALLENGER'):
    
    host = '127.0.0.1'
    port = '3306'
    sid = 'League Of Legends'

    user = 'root'
    password = 'tmddk720'
    
    con = pymysql.connect(host=host, user=user, password=password, db=sid, charset='utf8')
    sql = """SELECT gameId FROM matchlists JOIN league_queue ON matchlists.summonerId = league_queue.summonerId
                WHERE queue = %s AND tier = '%s' ORDER BY timestamp DESC;""" %(queue, tier)

    try:
        with con.cursor() as cursor:
            cursor.execute(sql)
            fetch = cursor.fetchall()
            
            if len(fetch) == 0:
                gameId_list = np.nan
            else:
                gameId_list = [int(x[0]) for x in fetch]
                
    except Exception as e:
        print(e)
    finally:
        con.close()
    
    return gameId_list

## 티어별 소환사 리스트 저장

In [6]:
#challenger to master
ch_queue_req = requests.get(url + 'lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'+api)
gm_queue_req = requests.get(url + 'lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'+api)
ms_queue_req = requests.get(url + 'lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'+api)

challengers = json_normalize(ch_queue_req.json()['entries'])
challengers['tier'] = 'CHALLENGER'
grandmasters = json_normalize(gm_queue_req.json()['entries'])
grandmasters['tier'] = 'GRANDMASTER'
masters = json_normalize(ms_queue_req.json()['entries'])
masters['tier'] = 'MASTER'

In [7]:
ch_to_dia = pd.concat([challengers, grandmasters, masters], ignore_index=True)

In [8]:
#diamondI to diamondIV
dias = {}

for tier in ['I', 'II', 'III', 'IV']:
    print('==================', tier, '==================')
    dias[tier] = pd.DataFrame()
    
    page = 1
    
    while(True):
        dia_queue_req = requests.get(url + 'lol/league/v4/entries/RANKED_SOLO_5x5/DIAMOND/'+tier +
                                     '?page=' + str(page) + '&api_key='+api_key)
        
        #2분에 100개 요청 넘어가는 경우
        if dia_queue_req.status_code == 429:
            time.sleep(120)
            continue
            
        #더이상 데이터 페이지가 없는 경우
        if len(dia_queue_req.content) == 2:
            print(tier, page)
            break
        else:
            temp_dias = json_normalize(dia_queue_req.json())
            dias[tier] = dias[tier].append(temp_dias)
            page += 1
       
    ch_to_dia = pd.concat([ch_to_dia, dias[tier]], ignore_index=True)
    
ch_to_dia['update'] = datetime.datetime.now()

print('# of summoners:    ', len(ch_to_dia))
print('# of challengers:  ', len(challengers))
print('# of grandmasters: ', len(grandmasters))
print('# of masters:      ', len(masters))
print('# of diamond1:     ', len(dias['I']))
print('# of diamond2:     ', len(dias['II']))
print('# of diamond3:     ', len(dias['III']))
print('# of diamond4:     ', len(dias['IV']))
print()

ch_to_dia.tail()

================== I ==================
I 37
================== II ==================
II 64
================== III ==================
III 130
================== IV ==================
IV 375
# of summoners:     125778
# of challengers:   300
# of grandmasters:  700
# of masters:       1668
# of diamond1:      7278
# of diamond2:      12877
# of diamond3:      26333
# of diamond4:      76622



,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak,tier,leagueId,queueType,miniSeries.target,miniSeries.wins,miniSeries.losses,miniSeries.progress,update
125773,o7iA9EOqqHTKTRJter2kt64GMtzeyHzVvpsPpg1m5cmu3Sg,YxxxxxxxxyC,0,IV,106,76,False,False,False,False,DIAMOND,c127b080-4359-462b-91f0-6070f3b6442d,RANKED_SOLO_5x5,NaN,NaN,NaN,NaN,2020-09-07 21:26:11.023741
125774,Ddt8nRrIq1pSWLB_JhDdTPNzq8MiB5tOAZ3f1F3rnFlb8w,buff markersman,3,IV,132,115,False,False,False,False,DIAMOND,c6b80a17-e510-48a0-83dd-2fdbbb8ef45d,RANKED_SOLO_5x5,NaN,NaN,NaN,NaN,2020-09-07 21:26:11.023741
125775,ZY8BJSpxfufLY5W0ZyxoCSHE0qsn9KBPqYeWtJSysglk8ho,김화타,0,IV,166,152,False,False,False,False,DIAMOND,f21dfb7d-2426-4503-9a6a-3409bd139827,RANKED_SOLO_5x5,NaN,NaN,NaN,NaN,2020-09-07 21:26:11.023741
125776,JUlvU2yNiOSbkj_aattbfvFR817IhDeOhH5thPsi6NkRZk4,zao zi ge ge,9,IV,172,142,False,False,False,False,DIAMOND,bf409f7f-2fe6-4f1a-8ef2-bc70d3996fcc,RANKED_SOLO_5x5,NaN,NaN,NaN,NaN,2020-09-07 21:26:11.023741
125777,7hkWQ8mn89PKScd-P7aUeTiP4NaNNaCkAFGc4oL09nZkygc,예술적인 솔킬각,35,IV,324,278,False,False,False,False,DIAMOND,fffc1cf1-8c91-4f0a-ba64-a646e9266cb8,RANKED_SOLO_5x5,NaN,NaN,NaN,NaN,2020-09-07 21:26:11.023741


In [10]:
ch_to_dia.to_csv('./data/ch_to_dia.csv')

In [13]:
for i in range(len(ch_to_dia)):
    summonerName = ch_to_dia['summonerName'][i]
    summonerId = ch_to_dia['summonerId'][i]
    tier = ch_to_dia['tier'][i]
    leaguePoints = str(ch_to_dia['leaguePoints'][i])
    rank = ch_to_dia['rank'][i]
    wins = str(ch_to_dia['wins'][i])
    losses = str(ch_to_dia['losses'][i])
    update = datetime.datetime.now()

    sql = """INSERT INTO `League Of Legends`.league_queue 
                (summonerName, summonerId, tier, leaguePoints, `rank`, wins, losses, `update`) 
             VALUES (%s, %s, %s, %s, %s, %s, %s, %s)  
             ON DUPLICATE KEY UPDATE \
                summonerName = VALUES(summonerName), \
                tier = VALUES(tier), \
                leaguePoints = VALUES(leaguePoints), \
                `rank` = VALUES(`rank`), \
                wins = VALUES(wins), \
                losses = VALUES(losses), \
                `update` = VALUES(`update`);"""

    values = (summonerName, summonerId, tier, leaguePoints, rank, wins, losses, update)

    update_db(host, port, sid, user, password, sql, values)

## 티어별 소환사 데이터 저장

In [ ]:
for tier in ['DIAMOND']:
    
    print('==================', tier, '==================')
    league_queue_df = get_league_queue_from_db(tier)
    
    N = len(league_queue_df)
    i = 44697
    
    while(i<N):
        si = league_queue_df.summonerId.values[i]
        
        try:
            summoner_req = requests.get(url + 'lol/summoner/v4/summoners/' + si + api)
            summoner_df = json_normalize(summoner_req.json())
            print('{0}   {1:5d}: {2}'.format(datetime.datetime.now().strftime("%H:%M:%S"), i, summoner_df.name[0]))
            
            summonerName = summoner_df['name'][0]
            summonerId = summoner_df['id'][0]
            accountId = summoner_df['accountId'][0]
            puuid = summoner_df['puuid'][0]
            summonerLevel = str(summoner_df['summonerLevel'][0])
            profileIconId = str(summoner_df['profileIconId'][0])
            revisionDate = str(summoner_df['revisionDate'][0])
            update = datetime.datetime.now()

            sql = """INSERT INTO `League Of Legends`.summoners 
                        (summonerName, summonerId, accountId, puuid, summonerLevel, profileIconId, revisionDate, `update`) 
                     VALUES (%s, %s, %s, %s, %s, %s, %s, %s)  
                     ON DUPLICATE KEY UPDATE \
                        summonerName = VALUES(summonerName), \
                        accountId = VALUES(accountId), \
                        puuid = VALUES(puuid), \
                        summonerLevel = VALUES(summonerLevel), \
                        profileIconId = VALUES(profileIconId), \
                        revisionDate = VALUES(revisionDate), \
                        `update` = VALUES(`update`);"""

            values = (summonerName, summonerId, accountId, puuid, summonerLevel, profileIconId, revisionDate, update)

            update_db(host, port, sid, user, password, sql, values)
        
        
        except Exception as e:
            #2분에 100개 제한
            if summoner_req.status_code == 429:
                time.sleep(120)
                continue
            
            #없는 데이터
            if summoner_req.status_code == 404:
                print('{0}   {1} '.format(datetime.datetime.now().strftime("%H:%M:%S"), e))
                pass
            else:
                print('{0}   {1} '.format(datetime.datetime.now().strftime("%H:%M:%S"), e))
                time.sleep(5)
                continue

        i += 1

## 소환사별 매치 데이터 다운로드

In [24]:
for tier in ['MASTER']:
    
    print('==================', tier, '==================')
    league_queue_df = get_league_queue_from_db(tier)
    
    N = len(league_queue_df)
    i = N-1
    
#     passthrough = True
    
    #for summoner
    while(i>=0):
        
        #tier별 소환사 목록에서 summonerId, summonerName 얻기
        si = league_queue_df.summonerId.values[i]
        sn = league_queue_df.summonerName.values[i]
        
#         if(sn == 'Hayasaka 4i'):
#             passthrough = False
        
#         if(passthrough):
#             i += 1
#             continue
        
        #summonerId로 summoner 데이터 불러오기
        summoner_df = get_summoner_from_db(si)
        if len(summoner_df) == 0:
            i -= 1
            continue
            
        ai = summoner_df['accountId'][0]
        
        #summonerId로 db에 저장되어 있는 가장 최근 매치 데이터 불러오기
        recent_matchId = get_recent_matchId_in_db(si)
        
        #begin index
        bi = 0
        yn_search = True
        
        while(yn_search):
            
            try:
                matches_req = requests.get(url + 'lol/match/v4/matchlists/by-account/' + ai + api + '&season=13&beginIndex=' + str(bi*100))           

                #2분에 100개 요청 넘어가는 경우
                if matches_req.status_code == 429:
                    time.sleep(120)
                    continue

                #더이상 데이터 페이지가 없는 경우
                if len(matches_req.json()['matches']) == 0:
                    print(sn, bi)
                    break
                
                #데이터 페이지를 불러온 경우
                else:
                    matches_df = json_normalize(matches_req.json())
                    matches_df = json_normalize(matches_df['matches'][0]).drop(columns=['role', 'lane'])
                    
                    matches_df['summonerName'] = sn
                    matches_df['summonerId'] = si
                    matches_df['accountId'] = ai
                    matches_df['timestamp'] = matches_df['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000).strftime("%Y-%m-%d %H:%M:%S"))
                    
                    sql = """INSERT INTO `League Of Legends`.matchlists
                                (platformId, gameId, champion, queue, season, timestamp, summonerName, summonerId, accountId) 
                             VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)  
                             ON DUPLICATE KEY UPDATE 
                                platformId = VALUES(platformId), 
                                champion = VALUES(champion), 
                                queue = VALUES(queue),
                                season = VALUES(season),
                                timestamp = VALUES(timestamp),
                                summonerName = VALUES(summonerName);"""
                    values = list(matches_df.itertuples(index=False, name=None))
                    
                    #db에 데이터 업데이트
                    update_db(host, port, sid, user, password, sql, values, option='many')
                    bi += 1
                    
                    #데이터베이스에 기록된 경기까지 찾은 경우 더 이상 검색하지 않음
                    if(recent_matchId in matches_df['gameId'].values):
                        print(sn, bi)
                        yn_search = False
                    
            except Exception as e:
                #없는 데이터
                if matches_req.status_code == 404:
                    print('{0}   {1} '.format(datetime.datetime.now().strftime("%H:%M:%S"), e))
                    pass
                else:
                    print('{0}   {1} '.format(datetime.datetime.now().strftime("%H:%M:%S"), e))
                    time.sleep(5)
                    continue
        
        i -= 1

================== MASTER ==================
09:56:41   'matches' 


KeyboardInterrupt: 

In [10]:
def cleanse_match_data(match_df):
    
    gameId = match_df['gameId'][0]
    
    teams_df = json_normalize(match_df['teams'][0])
    ptcp_df = json_normalize(match_df['participants'][0])
    ptcp_idt_df = json_normalize(match_df['participantIdentities'][0])
    
    
    #=====================================================================================================
    # teams_df 정리
    # - teamId, win, firstBlood, firstTower, firstInhibitor, firstBaron, firstDragon, firstRiftHerald,
    #   towerKills, inhibitorKills, baronKills, vilemawKills, riftHeraldKills, dominionVictoryScore, ban
    #=====================================================================================================
    
    #ban 목록 정리
    bans = {}
    for t in [0, 1]:
        teamId = (t+1)*100
        bans[str(teamId)] = json_normalize(teams_df['bans'][t]).T
        bans[str(teamId)].rename(columns={0:'ban1', 1:'ban2', 2:'ban3', 3:'ban4', 4:'ban5'}, inplace=True)
        bans[str(teamId)]['teamId'] = teamId
    bans = pd.concat([bans['100'], bans['200']])
    bans = bans.loc['championId']

    teams_df = pd.merge(teams_df.drop(columns=['bans']), bans, on='teamId', how='outer')
    teams_df['gameId'] = gameId
    for c in teams_df.columns:
        teams_df[c] = teams_df[c].astype(str)
    
    
    #=====================================================================================================
    # participants_df 정리
    # - participantId, teamId, championId, spell, win, item, kda, damage, gold, xp, visionScore, ward, rune
    #=====================================================================================================
   
    #participants column 명칭 정리
    ptcp_df.columns = ptcp_df.columns.str.strip().str.replace('stats.', '')
    ptcp_df.columns = ptcp_df.columns.str.strip().str.replace('timeline.', '')
    ptcp_df.columns = ptcp_df.columns.str.strip().str.replace('.', ':')
#     ptcp_df.columns = ptcp_df.columns.str.strip().str.replace('0-end', '0-10')
#     ptcp_df.columns = ptcp_df.columns.str.strip().str.replace('10-end', '10-20')
#     ptcp_df.columns = ptcp_df.columns.str.strip().str.replace('20-end', '20-30')
#     ptcp_df.columns = ptcp_df.columns.str.strip().str.replace('30-end', '30-40')
#     ptcp_df.columns = ptcp_df.columns.str.strip().str.replace('40-end', '40-50')
#     ptcp_df.columns = ptcp_df.columns.str.strip().str.replace('50-end', '50-60')
    
    #sq에 삽입하기 위해 특정 column 명칭 양쪽으로 ₩ 붙이기
    cc = ptcp_df.filter(regex='Deltas').columns
    ccc = ['`' + c + '`' for c in cc]
    ptcp_df.rename(columns=dict(zip(cc,ccc)), inplace=True)
    
    #중복 column 제거
    ptcp_df = ptcp_df.loc[:, ~ptcp_df.columns.duplicated()]
    
    #participants에서 제거할 column list
    participants_delete_list = ['killingSprees', 'doubleKills', 'tripleKills', 
                               'quadraKills', 'pentaKills', 'combatPlayerScore', 'objectivePlayerScore', 
                               'totalPlayerScore', 'totalScoreRank', 'role', 'lane']
    for i in range(10):
        participants_delete_list.append('playerScore'+str(i))
        
    #participants에서 필요 없는 column 제거
    ptcp_df = ptcp_df[ptcp_df.columns.drop(participants_delete_list)]
    ptcp_df = ptcp_df[ptcp_df.columns.drop(list(ptcp_df.filter(regex='Diff')))]
    ptcp_df = ptcp_df[ptcp_df.columns.drop(list(ptcp_df.filter(regex='Deltas')))]
    
    ptcp_df['gameId'] = gameId
    for c in ptcp_df.columns:
        ptcp_df[c] = ptcp_df[c].astype(str)
    
    
    #=====================================================================================================
    # participant_identities_df 정리
    # - participantId, accountId, summonerName, summonerId
    #=====================================================================================================
   
    #participant identities column 명칭 정리
    ptcp_idt_df.columns = ptcp_idt_df.columns.str.strip().str.replace('player.', '')
    
    #participant identities에서 필요 없는 column 제거
    ptcp_idt_df = ptcp_idt_df[ptcp_idt_df.columns.drop(['platformId', 'currentPlatformId', 'currentAccountId',
                                                        'matchHistoryUri', 'profileIcon'])]
  
    ptcp_idt_df['gameId'] = gameId
    for c in ptcp_idt_df.columns:
        ptcp_idt_df[c] = ptcp_idt_df[c].astype(str)
   
    
    #=====================================================================================================
    # match_df 정리
    # - gameId, platformId, gameCreation, gameDuration, queueId, mapId, seasonId, gameVersion, gameType
    #=====================================================================================================
   
    match_df = match_df.drop(columns=['teams', 'participants', 'participantIdentities'])
    match_df['gameCreation'] = match_df['gameCreation'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000).strftime("%Y-%m-%d %H:%M:%S"))
    
    for c in match_df.columns:
        match_df[c] = match_df[c].astype(str)
   
    return match_df, teams_df, ptcp_df, ptcp_idt_df

In [11]:
gameId_list = get_gameId_list_from_db(queue='420', tier='GRANDMASTER')
gameId_list = list(set(gameId_list))

In [12]:
gameId_list.index(3837180896)

254725

In [ ]:
N = len(gameId_list)
i = 120000
print(N)

while(i<N):    
    try:
        gameId = gameId_list[i]
        match_req = requests.get(url + 'lol/match/v4/matches/' + str(gameId) + api)           

        #2분에 100개 요청 넘어가는 경우
        if match_req.status_code == 429:
            time.sleep(120)
            continue
    
        match_df = json_normalize(match_req.json())
        match_df, teams_df, ptcp_df, ptcp_idt_df = cleanse_match_data(match_df)
           
        #match_df    
        sql = "INSERT INTO `League Of Legends`.`match` (" +  ', '.join(match_df.columns) + ") " + \
            "VALUES (" + ', '.join('%s' for i in match_df.columns) + ") ON DUPLICATE KEY UPDATE " + \
            ', '.join('{0}=VALUES({0})'.format(c) for c in match_df.drop(columns=['gameId']).columns) + ';'
        values = list(match_df.itertuples(index=False, name=None))[0]
        update_db(host, port, sid, user, password, sql, values)

        #teams_df
        sql = "INSERT INTO `League Of Legends`.`match_teams` (" +  ', '.join(teams_df.columns) + ") " + \
            "VALUES (" + ', '.join('%s' for i in teams_df.columns) + ") ON DUPLICATE KEY UPDATE " + \
            ', '.join('{0}=VALUES({0})'.format(c) for c in teams_df.drop(columns=['gameId', 'teamId']).columns) + ';'
        values = list(teams_df.itertuples(index=False, name=None))
        update_db(host, port, sid, user, password, sql, values, 'many')
        
        #ptcp_df
        sql = "INSERT INTO `League Of Legends`.`match_participants` (" +  ', '.join(ptcp_df.columns) + ") " + \
            "VALUES (" + ', '.join('%s' for i in ptcp_df.columns) + ") ON DUPLICATE KEY UPDATE " + \
            ', '.join('{0}=VALUES({0})'.format(c) for c in ptcp_df.drop(columns=['gameId', 'participantId']).columns) + ';'
        values = list(ptcp_df.itertuples(index=False, name=None))
        update_db(host, port, sid, user, password, sql, values, 'many')
    
        #ptcp_idt_df
        sql = "INSERT INTO `League Of Legends`.`match_participant_identities` (" +  ', '.join(ptcp_idt_df.columns) + ") " + \
            "VALUES (" + ', '.join('%s' for i in ptcp_idt_df.columns) + ") ON DUPLICATE KEY UPDATE " + \
            ', '.join('{0}=VALUES({0})'.format(c) for c in ptcp_idt_df.drop(columns=['gameId', 'participantId']).columns) + ';'
        values = list(ptcp_idt_df.itertuples(index=False, name=None))
        update_db(host, port, sid, user, password, sql, values, 'many')
    
        if i%100 == 0:
            print('{0}   {1:7d} - {2:10d}'.format(datetime.datetime.now().strftime("%H:%M:%S"), i, gameId))
        i += 1
    
    except Exception as e:
        #없는 데이터
        if match_req.status_code == 404 or match_req.status_code == 400:
            print('{0}   {1:10d} {2}'.format(datetime.datetime.now().strftime("%H:%M:%S"), gameId, e))
            break
        else:
            print('{0}   {1:10d} {2}'.format(datetime.datetime.now().strftime("%H:%M:%S"), gameId, e))
            time.sleep(5)
            continue
         

692021
02:18:55    120000 - 4226125208
02:19:39   4274359956 'gameId'
02:21:51    120100 - 3601174207
02:24:37    120200 - 3959787553
02:27:24    120300 - 4513435963
02:30:07    120400 - 3737490001
02:30:38   4398093056 'gameId'
02:33:02    120500 - 4584739669
02:35:51    120600 - 3550844060
02:38:34    120700 - 3515192791
02:41:19    120800 - 4249196309
02:44:03    120900 - 4406483000
02:46:46    121000 - 3957692776
02:49:32    121100 - 4393900627
02:52:17    121200 - 4159019895
02:55:03    121300 - 4427455645
02:57:49    121400 - 3645218311
02:58:19   4511342288 'gameId'
03:00:40    121500 - 4010123033
03:03:27    121600 - 3515195517
03:06:13    121700 - 3966083529
03:09:02    121800 - 3823477559
03:09:40   3559236647 'gameId'
03:11:55    121900 - 3966084181
03:14:38    122000 - 4498761077
(1366, "Incorrect integer value: 'nan' for column 'perkSubStyle' at row 8")
03:17:25    122100 - 3961890455
03:17:25   4041582232 'gameId'
03:17:59   3511003007 'gameId'
03:20:20    122200 - 434566